In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('netflix_titles.csv')

In [3]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [4]:
def create_textual_representation(row):
    textual_representation = f"""Type: {row['type']}
Title: {row['title']}
Director: {row['director']}
Cast: {row['cast']}
Country: {row['country']}
Release Year: {row['release_year']}
Rating: {row['rating']}
Duration: {row['duration']}
Description: {row['description']} """
    return textual_representation

In [5]:
df["textual_representation"]=df.apply(create_textual_representation, axis=1)

In [6]:
print(df["textual_representation"].head(5))

0    Type: Movie\nTitle: Dick Johnson Is Dead\nDire...
1    Type: TV Show\nTitle: Blood & Water\nDirector:...
2    Type: TV Show\nTitle: Ganglands\nDirector: Jul...
3    Type: TV Show\nTitle: Jailbirds New Orleans\nD...
4    Type: TV Show\nTitle: Kota Factory\nDirector: ...
Name: textual_representation, dtype: object


In [10]:
import ollama
response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': 'Hello!'}])
print(response['message']['content'])

Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


In [ ]:
import faiss
import numpy as np
import requests



dim =4096
index = faiss.IndexFlatL2(dim)

x=np.zeros((len(df['textual_representation']), dim), dtype='float32')

In [12]:
for i, representation in enumerate(df['textual_representation']):
    if i%200:
        print(f"Processing {i}th representation")
    
    res = requests.post("http://localhost:11434/api/embeddings",
                        json={'model':'llama3',
                            'prompt': representation,})
    
    embedding = res.json()['embedding']
    x[i] = np.array(embedding, dtype='float32')
index.add(x)
print("Indexing done")

Processing 1th representation
Processing 2th representation
Processing 3th representation
Processing 4th representation
Processing 5th representation
Processing 6th representation
Processing 7th representation
Processing 8th representation
Processing 9th representation
Processing 10th representation
Processing 11th representation
Processing 12th representation
Processing 13th representation
Processing 14th representation
Processing 15th representation
Processing 16th representation
Processing 17th representation
Processing 18th representation
Processing 19th representation
Processing 20th representation
Processing 21th representation
Processing 22th representation
Processing 23th representation
Processing 24th representation
Processing 25th representation
Processing 26th representation
Processing 27th representation
Processing 28th representation
Processing 29th representation
Processing 30th representation
Processing 31th representation
Processing 32th representation
Processing 33th r

In [13]:
faiss.write_index(index, 'index')

In [14]:
index = faiss.read_index('index')

In [21]:
df[df.title.str.contains('Inception')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,textual_representation
340,s341,Movie,Inception,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...","United States, United Kingdom","August 1, 2021",2010,PG-13,148 min,"Action & Adventure, Sci-Fi & Fantasy, Thrillers",A troubled thief who extracts secrets from peo...,Type: Movie\nTitle: Inception\nDirector: Chris...


In [22]:
favourite_movie = df.iloc[340]

In [32]:
res = requests.post("http://localhost:11434/api/embeddings",
                    json={'model':'llama3',
                        'prompt': favourite_movie['textual_representation'],})
embedding = res.json()['embedding']
embedding = np.array(embedding, dtype='float32')
D, I = index.search(embedding.reshape(1, -1), 5)
print("Top 5 similar movies:")
best_matches= np.array(df['textual_representation'])[I.flatten()]
for i, match in enumerate(best_matches):
    print(f"Match {i+1}: {match}")
    print("--------------------------------------------------")

Top 5 similar movies:
Match 1: Type: Movie
Title: Inception
Director: Christopher Nolan
Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Tom Hardy, Ken Watanabe, Dileep Rao, Cillian Murphy, Tom Berenger, Marion Cotillard, Pete Postlethwaite, Michael Caine, Lukas Haas
Country: United States, United Kingdom
Release Year: 2010
Rating: PG-13
Duration: 148 min
Description: A troubled thief who extracts secrets from people's dreams takes one last job: leading a dangerous mission to plant an idea in a target's subconscious. 
--------------------------------------------------
Match 2: Type: Movie
Title: The Last Days of American Crime
Director: Olivier Megaton
Cast: Edgar Ramírez, Michael Pitt, Anna Brewster, Patrick Bergin, Sharlto Copley, Brandon Auret, Tamer Burjaq, Terence Maynard, James Richard Marshall
Country: United States
Release Year: 2020
Rating: TV-MA
Duration: 149 min
Description: A bank robber joins a plot to commit one final, historic heist before the government turns

In [37]:
representation="""Type: Movie
Title: The White Tiger
Director: Ramin Bahrani
Cast: Priyanka Chopra, Rajkummar Rao, Adarsh Gourav, Mahesh Manjrekar, Swaroop Sampat, Daya Shankar Pandey, Aditi Sudhakar
Country: India
Release Year: 2021
Rating: R
Duration: 125 min 
"""

In [ ]:
res = requests.post("http://localhost:11434/api/embeddings",
                    json={'model':'llama3',
                        'prompt': representation})
embedding = res.json()['embedding']
embedding = np.array(embedding, dtype='float32')
D, I = index.search(embedding.reshape(1, -1), 5)
print("Top 5 similar movies:")
best_matches= np.array(df['textual_representation'])[I.flatten()]
for i, match in enumerate(best_matches):   
    print(f"Match {i+1}: {match}")
    print("---------  -----------------------------------------")

Top 5 similar movies:
Match 1: Type: Movie
Title: Tell Me Who I Am
Director: Ed Perkins
Cast: nan
Country: United Kingdom
Release Year: 2019
Rating: TV-MA
Duration: 86 min
Description: In this documentary, Alex trusts his twin, Marcus, to tell him about his past after he loses his memory. But Marcus is hiding a dark family secret. 
--------------------------------------------------
Match 2: Type: Movie
Title: Five Nights in Maine
Director: Maris Curran
Cast: David Oyelowo, Dianne Wiest, Rosie Perez, Teyonah Parris, Hani Furstenberg, Bill Raymond
Country: United States
Release Year: 2015
Rating: TV-MA
Duration: 83 min
Description: After his wife dies in a car accident, a grief-stricken man visits his estranged mother-in-law in Maine, where they try to help each other heal. 
--------------------------------------------------
Match 3: Type: Movie
Title: Anarkali of Aarah
Director: Avinash Das
Cast: Swara Bhaskar, Sanjay Mishra, Vijay Kumar, Pankaj Tripathi, Mayur More, Ishtiyak Khan
Count